In [2]:
#import library
import requests
import pandas as pd
import tensorflow as tf
import json
import base64

In [3]:
#laod dataset yang akan digunakan untuk test
data = pd.read_csv('data/Stress.csv')

In [30]:
data['text'][1999]

'We will be using THE RESPECTFUL PROSTITUTE by Jean Paul Sartre This is to be one presentation that includes all of these elements. I HIGHLY recommend a PowerPoint. Take a good, clear picture or scan of your ground plan and sketch and add those as slides. Organization of your presentation is important.'

In [28]:
test1 = data['text'][1199]

In [32]:
def prepare_json(text):
    feature_spec = {
        "text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")])),
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

json_data = prepare_json(test1)
    
endpoint = "http://localhost:8501/v1/models/stress-prediction-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
if prediction:
    prediction_value = prediction[0][0]
    result = "Stress" if prediction_value > 0.5 else "Tidak Stress"
else:
    result = "Error: No predictions found in response."

print(result)

Stress


Apabila Prediksi lebih besar dari 0.5 maka akan menghasilkan prediksi dari text "Stress", apabila dibawah 0.5 maka tidak stress